# Introduction à l'apprentissage automatique: complément facultatif <font color=red> - CORRECTION </font>

<br>

## Complément facultatif: génération aléatoire d'images de chiffres manuscrits selon un mélange de gaussiennes


On change de base de données pour utiliser:
```python
digits = datasets.load_digits()
X_train2 = digits.data
y_train2 = digits.target 
``` 
Cette base est formée de 1797 images de taille 8 pixels par 8 pixels. La dimension des observations (64) est donc bien plus faible que dans la base MNIST, ce qui rend les calculs moins longs et les modèles moins sujets à la malédiction de la dimension.

Modélisez la distribution des chiffres manuscrits comme un mélange de gaussienne:
* vous utiliserez un nombre de composantes minimisant le critère AIC (testez entre 1 et 200 composantes par pas de 20 pour limiter le temps de calcul)
* vous représenterez les moyennes des composantes comme une image, en vous inspirant de la fonction d'affichage précédente (constatez que certains chiffres sont représentés par plusieurs gaussiennes)
* vous générerez aléatoirement 100 chiffres avec cette méthode. Vous les représenterez, et vous les comparerez à 100 chiffres de la base.


Si vous avez des résultats intéressants, n'hésitez pas à discuter vos résultats avec votre chargé de TD. 

In [ ]:
from sklearn import mixture, datasets
import numpy as np
import matplotlib.pyplot as plt

digits = datasets.load_digits()
X_train2 = digits.data
y_train2 = digits.target
print(len(X_train2))
size_images=(8,8)

tab_AIC = []
for K in range(1,201,20):
    print("\nnombre composantes: %d" %K)
    GMM = mixture.GaussianMixture(n_components=K, covariance_type="full")
    %time GMM.fit(X_train2)
    AIC = GMM.aic(X_train2)
    tab_AIC.append(AIC)
    print("AIC: %d" %AIC)

tab_K=np.array(range(1,201,20))
plt.figure()
plt.plot(tab_K,tab_AIC)
print("valeur de K minimisant AIC (à 20 près): K=%d" %tab_K[np.argmin(tab_AIC)])

In [ ]:
K=120  # K réalisant le minimum de AIC (ou K=100, selon les exécutions)
# faites aussi l'expérience avec K=10: chaque chiffre est représenté par une classe du mélange (voir les moyennes),
# mais la génération aléatoire est bien moins réaliste car chaque chiffre ne peut pas être représenté de 
# manière satisfaisante par une unique gaussienne


GMM=mixture.GaussianMixture(n_components=K, covariance_type='full')  
%time GMM.fit(X_train2)  # EM

# Représentation des moyennes du mélange:
plt.figure(figsize=[15,12])    
for n in range(K):
   plt.subplot(10,12,n+1,xticks=[],yticks=[])
   plt.imshow(np.reshape(GMM.means_[n,:],size_images),cmap='gray_r',vmin=0,vmax=16)
plt.suptitle('moyennes des composantes du mélange de gaussiennes')
plt.show();

In [ ]:

# 100 chiffres générés aléatoirement:
%time data_new_X, data_new_y = GMM.sample(100)
plt.figure(figsize=[15,12])    
for n in range(100):
   plt.subplot(10,10,n+1,xticks=[],yticks=[])
   plt.imshow(np.reshape(data_new_X[n,:],size_images),cmap='gray_r',vmin=0,vmax=16)
   #plt.text(0.1,0.1,data_new_y[n],fontsize=6,bbox=dict(facecolor='white', alpha=1))
plt.suptitle('100 chiffres synthétisés aléatoirement')
plt.show()

# à comparer aux 100 premiers chiffres de la base:
plt.figure(figsize=[15,12])    
for n in range(100):
   plt.subplot(10,10,n+1,xticks=[],yticks=[])
   plt.imshow(np.reshape(X_train2[n,:],size_images),cmap='gray_r',vmin=0,vmax=16)
plt.suptitle('les 100 premiers chiffres issus de la base de données (pour comparaison)')
plt.show();

# Les chiffres synthétiques générés à partir d'un mélange de 120 gaussiennes ressemblent 
# plutôt bien aux vrais chiffres


In [ ]:
# On peut pousser la vérification du réalisme de la synthèse de chiffres 
# en faisant une classification des chiffres synthétiques 
# (par exemple avec KNN) et vérifier si la classe identifiée est plausible:

from sklearn import neighbors
import numpy as np
import matplotlib.pyplot as plt

# (on adapte affichage_150_images du TD précédent)
def affichage_100_images(X_test,y_pred):
    plt.figure(figsize=[15,12])   
    for n in range(100):
        plt.subplot(10,10,n+1,xticks=[],yticks=[])
        plt.imshow(np.reshape(X_test[n,:],size_images),cmap='gray_r',vmin=0,vmax=16)
        plt.text(0.1,0.1,str(y_pred[n]),fontsize=8,bbox=dict(facecolor='red', alpha=1))    
    plt.suptitle('classe predite / classe réelle')
    plt.show();
    
# classification au plus proche voisin et affichage
knn = neighbors.KNeighborsClassifier(n_neighbors=1, n_jobs=12) 
knn.fit(X_train2, y_train2)
y_pred_nn = knn.predict(data_new_X)
affichage_100_images(data_new_X,y_pred_nn)
# le résultat est plutôt convaincant